### Tennis court keypoint model
In order to track player 1, player 2 and ball positions, we could train a model to identify keypoints on the tennis court, and then use them as a reference to track their movement data along the court. Our inspiration comes from an amazing deep learning project with the same purpose, proposed by [Yastrebksv](https://github.com/yastrebksv/TennisCourtDetector). We are also using his dataset to train our keypoint model.

![image](images/tennis_court_keypoints_example.png)


### 1 - Download tennis court keypoint dataset

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu' -O 'tennis_court_dataset.zip'
!unzip tennis_court_dataset.zip

### 2 - Download and preprocessing our dataset
In order to work with our data, we will set a transformation pipeline 

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [ ]:
# set cuda as device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Create Torch Dataset
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            # ImageNet's normalization pattern for pre-trained models
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(items['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [ ]:
# Set training and validation datasets
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset = KeypointsDataset("data/images", "data/data_val.json") # Check if it is the right json file here

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

### 3 - Training our model

In [ ]:
# Set the model for training 
model = models.resnet50(pretrained=True)

# Fine-tuning
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)

# Move model to cuda
model = model.to(device)

# Define MSELoss as loss function
criterion = torch.nn.MSELoss()

# Define optimizer as Adam
lr = 1e-4 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

epochs = 20

for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps - kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch: {epoch}, iter: {i}, loss: {loss.item()}")


In [ ]:
# Download and save our model
torch.save(model.state_dict(), "keypoints_model.pth")